In [18]:
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import degree
from collections import Counter
import matplotlib.pyplot as plt
import networkx as nx 
from torch_geometric.utils import to_networkx
import pandas as pd
import numpy as np

In [19]:
# loading cora dataset
dataset = Planetoid(root=".", name="Cora")
# access the first graph
data = dataset[0]

print(f'Dataset: {dataset}')
print(f'-------------------')
print(f'Number of graphs: {len(dataset)}')
print(f'{data}')
print(f'Number of nodes: {data.x.shape[0]}') # using shape to check dimenstion row is nodes and col is features
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')


print(f'Graph:')
print(f'-------------------')
print(f'Edges are directed: {data.is_directed()}')
print(f'Graph has isolated nodes: {data.has_isolated_nodes()}')
print(f'Graph has loops: {data.has_self_loops()}')


print(f'-------------------')
print(f'Tensor:')
print(f'Features of first 3 nodes (publication papers)\n {data.x[:3]}')
print(f'First 5 edges\n {data.edge_index[:, :5]}')
print(f'label of 5 node\n {data.y[:5]}')


Dataset: Cora()
-------------------
Number of graphs: 1
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of nodes: 2708
Number of features: 1433
Number of classes: 7
Graph:
-------------------
Edges are directed: False
Graph has isolated nodes: False
Graph has loops: False
-------------------
Tensor:
Features of first 3 nodes (publication papers)
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
First 5 edges
 tensor([[   0,    0,    0,    1,    1],
        [ 633, 1862, 2582,    2,  652]])
label of 5 node
 tensor([3, 4, 4, 0, 3])


## Inspect data as tabular data

In [20]:
df = pd.DataFrame(data.x.numpy())
df['label'] = pd.DataFrame(data.y)
df

,0,1,2,3,4,5,6,7,8,9,...,1424,1425,1426,1427,1428,1429,1430,1431,1432,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2704,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


## Vanila neural nets

In [21]:
class_labels = [
    'Case Based',
    'Genetic Algorithms',
    'Neural Networks',
    'Probabilistic Methods',
    'Reinforcement Learning',
    'Rule Learning',
    'Theory'
]

In [22]:
import torch
from torch.nn import Linear
import torch.nn.functional as F

# Define accuracy metric as the number of correctly predicted results / total numbe of samples
def accuracy(y_pred, y_true):
    return torch.sum(y_pred == y_true) / len(y_true)

# The artificial neural netwrok without topological data
class MLP(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.linear1 = Linear(dim_in, dim_h) # input layer -> hidden layer
        self.linear2 = Linear(dim_h, dim_out) # hidden layer -> output layer

    def forward(self, x):
        x = self.linear1(x)
        x = torch.relu(x)
        x = self.linear2(x)
        return F.log_softmax(x, dim=1) # Softmax layer to convert output to probability between 0 and 1
    
    # Training function
    def fit(self, data, epochs):
        criterion = torch.nn.CrossEntropyLoss() # Use cross entropy loss as loss function
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4) # Use Adam optimizer

        # main training loop
        self.train() # Set the model in tranining mode
        for epoch in range(epochs+1):
            optimizer.zero_grad() # Clear the gradients
            out = self(data.x)
            loss = criterion(out[data.train_mask], data.y[data.train_mask]) # calculate the loss between output and ground truth
            acc = accuracy(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
            loss.backward() # Back propogate the loss
            optimizer.step() # Gradient descent

            # Print the stats every 20 epochs
            if epoch % 20 == 0:
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
                val_acc = accuracy(out[data.val_mask].argmax(dim=1), data.y[data.val_mask])
                print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: {acc*100:>5.2f}% | Val Loss: {val_loss:.2f} | Val Acc: {val_acc*100:.2f}%')

    def test(self, data):
        self.eval()
        out = self(data.x)
        acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
        return acc

In [23]:
mlp = MLP(dataset.num_features, 16, dataset.num_classes)
print(mlp)

MLP(
  (linear1): Linear(in_features=1433, out_features=16, bias=True)
  (linear2): Linear(in_features=16, out_features=7, bias=True)
)


In [24]:
mlp.fit(data, epochs=100)

Epoch   0 | Train Loss: 1.958 | Train Acc: 15.00% | Val Loss: 1.92 | Val Acc: 12.20%
Epoch  20 | Train Loss: 0.113 | Train Acc: 100.00% | Val Loss: 1.43 | Val Acc: 50.40%
Epoch  40 | Train Loss: 0.014 | Train Acc: 100.00% | Val Loss: 1.51 | Val Acc: 50.60%
Epoch  60 | Train Loss: 0.007 | Train Acc: 100.00% | Val Loss: 1.47 | Val Acc: 51.80%
Epoch  80 | Train Loss: 0.008 | Train Acc: 100.00% | Val Loss: 1.40 | Val Acc: 52.00%
Epoch 100 | Train Loss: 0.008 | Train Acc: 100.00% | Val Loss: 1.37 | Val Acc: 52.00%


In [25]:
acc = mlp.test(data)
print(f'MLP test accuracy: {acc*100:.2f}%')

MLP test accuracy: 53.50%


## Graph Neural Network

In [26]:
# Define the layer of GNN
class VanillaGNNLayer(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.linear = Linear(dim_in, dim_out, bias=False)

    def forward(self, x, adjacency):
        x = self.linear(x)
        x = torch.sparse.mm(adjacency, x) # Matrix multiplication on ajacency matrix to accumulate the value of neighbor nodes
        return x

In [27]:
from torch_geometric.utils import to_dense_adj

adjacency = to_dense_adj(data.edge_index)[0]
adjacency += torch.eye(len(adjacency))
adjacency

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 1.],
        [0., 0., 0.,  ..., 0., 1., 1.]])

In [28]:
class VanillaGNN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gnn1 = VanillaGNNLayer(dim_in, dim_h) # Use our customized layer for forwarding the data
        self.gnn2 = VanillaGNNLayer(dim_h, dim_out)

    def forward(self, x, adjacency):
        h = self.gnn1(x, adjacency)
        h = torch.relu(h)
        h = self.gnn2(h, adjacency)
        return F.log_softmax(h, dim=1)

    def fit(self, data, epochs):
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)
        self.train()
        for epoch in range(epochs+1):
            optimizer.zero_grad()
            out = self(data.x, adjacency)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            acc = accuracy(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
            loss.backward()
            optimizer.step()
            if epoch % 20 == 0:
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
                val_acc = accuracy(out[data.val_mask].argmax(dim=1), data.y[data.val_mask])
                print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: {acc*100:>5.2f}% | Val Loss: {val_loss:.2f} | Val Acc: {val_acc*100:.2f}%')
                
    def test(self, data):
        self.eval()
        out = self(data.x, adjacency)
        acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
        return acc

In [29]:
gnn = VanillaGNN(dataset.num_features, 16, dataset.num_classes)
print(gnn)
gnn.fit(data, epochs=100)
acc = gnn.test(data)
print(f'\nGNN test accuracy: {acc*100:.2f}%')

VanillaGNN(
  (gnn1): VanillaGNNLayer(
    (linear): Linear(in_features=1433, out_features=16, bias=False)
  )
  (gnn2): VanillaGNNLayer(
    (linear): Linear(in_features=16, out_features=7, bias=False)
  )
)
Epoch   0 | Train Loss: 2.168 | Train Acc: 12.86% | Val Loss: 2.17 | Val Acc: 9.00%
Epoch  20 | Train Loss: 0.156 | Train Acc: 97.86% | Val Loss: 1.33 | Val Acc: 73.80%
Epoch  40 | Train Loss: 0.027 | Train Acc: 99.29% | Val Loss: 1.86 | Val Acc: 72.80%
Epoch  60 | Train Loss: 0.006 | Train Acc: 100.00% | Val Loss: 2.06 | Val Acc: 72.60%
Epoch  80 | Train Loss: 0.003 | Train Acc: 100.00% | Val Loss: 2.08 | Val Acc: 72.00%
Epoch 100 | Train Loss: 0.003 | Train Acc: 100.00% | Val Loss: 2.09 | Val Acc: 72.60%

GNN test accuracy: 74.20%


## Graph Convolutional Neural Network (GCN)

In [30]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

def accuracy(pred_y, y):
    return ((pred_y == y).sum() / len(y)).item()

class GCN(torch.nn.Module):
    """Graph Convolutional Network"""
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gcn1 = GCNConv(dim_in, dim_h) # Use the GCNConv layer from torch_geometric library
        self.gcn2 = GCNConv(dim_h, dim_out)

    def forward(self, x, edge_index):
        h = self.gcn1(x, edge_index)
        h = torch.relu(h)
        h = self.gcn2(h, edge_index)
        return F.log_softmax(h, dim=1)
    
    def fit(self, data, epochs):
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(),
                                    lr=0.01,
                                    weight_decay=5e-4)
        self.train()
        for epoch in range(epochs+1):
            optimizer.zero_grad()
            out = self(data.x, data.edge_index)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            acc = accuracy(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
            loss.backward()
            optimizer.step()
            if(epoch % 20 == 0):
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
                val_acc = accuracy(out[data.val_mask].argmax(dim=1), data.y[data.val_mask])
                print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: {acc*100:>5.2f}% | Val Loss: {val_loss:.2f} | Val Acc: {val_acc*100:.2f}%')

    @torch.no_grad()
    def test(self, data):
        self.eval()
        out = self(data.x, data.edge_index)
        acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
        return acc

In [31]:
gcn = GCN(dataset.num_features, 16, dataset.num_classes)
print(gcn)
gcn.fit(data, epochs=100)

GCN(
  (gcn1): GCNConv(1433, 16)
  (gcn2): GCNConv(16, 7)
)
Epoch   0 | Train Loss: 1.951 | Train Acc: 10.71% | Val Loss: 1.97 | Val Acc: 11.20%
Epoch  20 | Train Loss: 0.107 | Train Acc: 100.00% | Val Loss: 0.83 | Val Acc: 76.80%
Epoch  40 | Train Loss: 0.013 | Train Acc: 100.00% | Val Loss: 0.80 | Val Acc: 77.00%
Epoch  60 | Train Loss: 0.013 | Train Acc: 100.00% | Val Loss: 0.77 | Val Acc: 77.20%
Epoch  80 | Train Loss: 0.016 | Train Acc: 100.00% | Val Loss: 0.75 | Val Acc: 77.00%
Epoch 100 | Train Loss: 0.015 | Train Acc: 100.00% | Val Loss: 0.75 | Val Acc: 77.00%


In [32]:
acc = gcn.test(data)
print(f'GCN test accuracy: {acc*100:.2f}%')

GCN test accuracy: 80.30%
